## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ugoofaaru,MV,2022-02-03 22:54:25,5.6667,73.0000,80.13,76,14,5.50,few clouds
1,1,Ilulissat,GL,2022-02-03 22:52:34,69.2167,-51.1000,17.78,70,100,9.91,overcast clouds
2,2,Butaritari,KI,2022-02-03 22:50:30,3.0707,172.7902,82.11,74,24,15.32,light rain
3,3,Hermanus,ZA,2022-02-03 22:54:25,-34.4187,19.2345,62.80,71,39,9.60,scattered clouds
4,4,Torbay,CA,2022-02-03 22:50:29,47.6666,-52.7314,35.58,84,100,28.77,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ugoofaaru,MV,2022-02-03 22:54:25,5.6667,73.0000,80.13,76,14,5.50,few clouds
2,2,Butaritari,KI,2022-02-03 22:50:30,3.0707,172.7902,82.11,74,24,15.32,light rain
7,7,Vaini,TO,2022-02-03 22:49:40,-21.2000,-175.2000,86.16,79,40,8.05,scattered clouds
9,9,Avarua,CK,2022-02-03 22:51:38,-21.2078,-159.7750,84.25,74,40,18.41,scattered clouds
10,10,Kahului,US,2022-02-03 22:54:27,20.8947,-156.4700,83.46,59,2,11.01,clear sky
14,14,Biak,ID,2022-02-03 22:54:28,-0.9131,122.8766,77.56,74,100,3.60,light rain
15,15,Non Thai,TH,2022-02-03 22:54:29,15.1981,102.0358,75.85,68,62,8.46,broken clouds
18,18,Bambous Virieux,MU,2022-02-03 22:50:29,-20.3428,57.7575,80.85,89,75,25.32,moderate rain
23,23,Kapaa,US,2022-02-03 22:54:30,22.0752,-159.3190,78.48,59,2,1.99,clear sky
26,26,Manggar,ID,2022-02-03 22:49:25,-2.8833,108.2667,77.38,86,76,3.76,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count().isna()

City_ID        False
City           False
Country        False
Date           False
Lat            False
Lng            False
Max Temp       False
Humidity       False
Cloudiness     False
Wind Speed     False
Description    False
dtype: bool

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ugoofaaru,MV,80.13,few clouds,5.6667,73.0000,
2,Butaritari,KI,82.11,light rain,3.0707,172.7902,
7,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,
9,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,
10,Kahului,US,83.46,clear sky,20.8947,-156.4700,
14,Biak,ID,77.56,light rain,-0.9131,122.8766,
15,Non Thai,TH,75.85,broken clouds,15.1981,102.0358,
18,Bambous Virieux,MU,80.85,moderate rain,-20.3428,57.7575,
23,Kapaa,US,78.48,clear sky,22.0752,-159.3190,
26,Manggar,ID,77.38,broken clouds,-2.8833,108.2667,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ugoofaaru,MV,80.13,few clouds,5.6667,73.0000,Ungoofaaru Inn
2,Butaritari,KI,82.11,light rain,3.0707,172.7902,Isles Sunset Lodge
7,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Kahului,US,83.46,clear sky,20.8947,-156.4700,Maui Seaside Hotel
...,...,...,...,...,...,...,...
687,Hobyo,SO,75.96,scattered clouds,5.3505,48.5268,Halane Hotel and Restaurent
689,Gladstone,AU,78.78,overcast clouds,-23.8500,151.2500,Highpoint International Hotel
691,Anloga,GH,80.94,clear sky,5.7947,0.8973,Pin Drop Hotel
692,Loandjili,CG,78.28,overcast clouds,-4.7561,11.8578,Résidence ARCHANGE


In [22]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))